In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import pylab as plt

from read_dataset import read_mfcc_10frame_with_train_test
from read_saved_models import loadMfcc10frameStanderdScaler

Using TensorFlow backend.


In [2]:
def getStanderizedData(data):
    data_shape = data.shape
    n = data_shape[0]
    reshaped_data = data.reshape(n, -1)
    saved_ss = loadMfcc10frameStanderdScaler()
    trasformed_data = saved_ss.transform(reshaped_data)
    ret_data = trasformed_data.reshape(data_shape)
    return ret_data

In [3]:
X_train, X_test, y_train, y_test = read_mfcc_10frame_with_train_test()
X_ss_train = getStanderizedData(X_train)
X_ss_test = getStanderizedData(X_test)

In [4]:
print(X_ss_train.shape)

(800, 1, 10, 128, 128)


In [7]:
seq = Sequential()
seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   input_shape=(1, 10, 128, 128),
                   padding='same', return_sequences=True, activation='relu', data_format='channels_first'))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(Flatten())
seq.add(Dense(100, activation='relu'))
seq.add(Dropout(0.4))
seq.add(BatchNormalization())

seq.add(Dense(10, activation='softmax'))

seq.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy', 'categorical_accuracy'])


In [8]:
result = seq.fit(X_ss_train, y_train, batch_size=10,
        epochs=300, validation_split=0.1)

Train on 720 samples, validate on 80 samples
Epoch 1/300
720/720 [==============================] - 2547s - loss: 0.3005 - acc: 0.8992 - categorical_accuracy: 0.3194 - val_loss: 0.3308 - val_acc: 0.9000 - val_categorical_accuracy: 0.1000
Epoch 2/300
720/720 [==============================] - 2528s - loss: 0.1538 - acc: 0.9387 - categorical_accuracy: 0.6792 - val_loss: 0.3319 - val_acc: 0.9000 - val_categorical_accuracy: 0.1000
Epoch 3/300
640/720 [=========================>....] - ETA: 266s - loss: 0.0975 - acc: 0.9664 - categorical_accuracy: 0.8500

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/minato/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-aa26d0b794e9>", line 2, in <module>
    epochs=300, validation_split=0.1)
  File "/home/minato/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/keras/models.py", line 870, in fit
    initial_epoch=initial_epoch)
  File "/home/minato/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/keras/engine/training.py", line 1507, in fit
    initial_epoch=initial_epoch)
  File "/home/minato/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/keras/engine/training.py", line 1156, in _fit_loop
    outs = f(ins_batch)
  File "/home/minato/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py", line 2269, in __call__
    **self.session_kwargs)
  File "/home/minato/.pyenv/versions/ana

KeyboardInterrupt: 